In [ ]:
!pip install transformers
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd

train_df = pd.read_csv('non_skewed_train.csv')
test_df = pd.read_csv('non_skewed_test.csv')
df = pd.concat([train_df, test_df])
df

,text,label
0,<user> <user> go home drunk <user> maga trump2...,1
1,amazon investigating chinese employees selling...,0
2,<user> canada need another cuck already enough...,1
3,<user> <user> <user> <user> <user> <user> <use...,0
4,maga <user> <emoji : musical_notes> sing like ...,0
...,...,...
3312,<user> <user> cool 110 deserves recognition ge...,0
3313,<user> maga kag winning,0
3314,rt <user> hoe bust ha knee caps 128128 128128 ...,1
3315,<user> speaking,0


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df

,text,label
0,niggas say like latinas mean fat bitches smell...,1
1,<user> <user> <user> sorry hear,0
2,rt <user> would never trip bitch aint mine 128...,1
3,<user> <user> <user> <user> <user> <user> <use...,0
4,<user> happy birthday mrs aggers hope great da...,0
...,...,...
15473,<user> <user> front front see love know happen...,0
15474,<user> <user> mk ultra,0
15475,<user> never one much rule law law rule,0
15476,<user> haha great ending modern antifa embarra...,0


In [ ]:
test_df = df.sample(frac = 0.2, random_state = 42)
train_df = df.drop(test_df.index)

In [ ]:
train_df.to_csv('non_skewed_train.csv', index = False)
test_df.to_csv('non_skewed_test.csv', index = False)

In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'non_skewed_train.csv', 'test': 'non_skewed_test.csv'})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 12382
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3096
    })
})

In [ ]:
dataset['train'][1]

{'text': 'rt <user> would never trip bitch aint mine 128514 128514 niggas clowns man',
 'label': 1}

In [ ]:
dataset['train'].features.keys()

dict_keys(['text', 'label'])

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length = 30)
  return encoding

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/12382 [00:00<?, ? examples/s]

Map:   0%|          | 0/3096 [00:00<?, ? examples/s]

In [ ]:
train_ds = encoded_dataset["train"].shuffle(seed=42)
val_ds = encoded_dataset["test"].shuffle(seed=42)

In [ ]:
train_ds.set_format("torch")
val_ds.set_format("torch")

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis",
                                                           num_labels=2, ignore_mismatched_sizes=True)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at finiteautomata/bertweet-base-sentiment-analysis and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
import accelerate

accelerate.__version__

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bertweet-olid",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjusted batch size
    per_device_eval_batch_size=8,  # Adjusted batch size
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    gradient_accumulation_steps=2,  # Gradient accumulation
    logging_steps=100,  # Adjust as needed
    eval_steps=200,  # Adjust as needed
)

In [ ]:
!pip install evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AdamW, get_scheduler
optimizer = AdamW(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
scheduler = get_scheduler(
    "linear",
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_ds) // args.gradient_accumulation_steps * args.num_train_epochs,
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, tuple):
      logits = logits[0]
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(predictions, labels)
    f1 = f1_score(predictions, labels, average = 'macro')

    return {"accuracy": acc, "f1": f1}

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.367500,0.425681,0.823320,0.821724
2,0.300600,0.365094,0.838501,0.838346
3,0.246100,0.481162,0.830103,0.829839
4,0.202000,0.683489,0.832364,0.832364
5,0.153300,1.030289,0.810401,0.808208


TrainOutput(global_step=3870, training_loss=0.2500755724056747, metrics={'train_runtime': 698.2169, 'train_samples_per_second': 88.669, 'train_steps_per_second': 5.543, 'total_flos': 954445631094000.0, 'train_loss': 0.2500755724056747, 'epoch': 5.0})

In [ ]:
preds = trainer.predict(val_ds)

In [ ]:
val_ds[0]

{'text': '<user> <user> yup basically another dead cat url',
 'label': tensor(0),
 'input_ids': tensor([    0, 10584, 56843,   241, 10584, 56843,   241,  6245,  2224,   347,
           839,  1407, 20922,     2,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0])}

In [ ]:
preds

PredictionOutput(predictions=array([[ 0.23431817,  0.0100347 ],
       [-0.9520443 ,  1.1749288 ],
       [-2.6940582 ,  2.6599271 ],
       ...,
       [ 0.836186  , -0.47527888],
       [-2.6686673 ,  2.6592047 ],
       [-2.3942583 ,  2.4176505 ]], dtype=float32), label_ids=array([0, 1, 1, ..., 0, 1, 1]), metrics={'test_loss': 0.3650943636894226, 'test_accuracy': 0.838501291989664, 'test_f1': 0.8383458646616542, 'test_runtime': 6.2967, 'test_samples_per_second': 491.686, 'test_steps_per_second': 61.461})

In [ ]:
pred_vals = np.argmax(preds.predictions, axis=-1)
labels = preds.label_ids
pred_vals, labels

(array([0, 1, 1, ..., 0, 1, 1]), array([0, 1, 1, ..., 0, 1, 1]))

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
print('roc: ', roc_auc_score(labels, pred_vals))
print('f1: ', f1_score(labels, pred_vals, average = 'macro'))
print('accuracy: ', accuracy_score(labels, pred_vals))

roc:  0.8382680697546127
f1:  0.8383458646616542
accuracy:  0.838501291989664


In [ ]:
trainer.save_model("./bert_tweet")

In [ ]:
!zip -r bert_tweet.zip bert_tweet/

  adding: bert_tweet/ (stored 0%)
  adding: bert_tweet/training_args.bin (deflated 51%)
  adding: bert_tweet/added_tokens.json (stored 0%)
  adding: bert_tweet/vocab.txt (deflated 51%)
  adding: bert_tweet/model.safetensors (deflated 9%)
  adding: bert_tweet/bpe.codes (deflated 58%)
  adding: bert_tweet/special_tokens_map.json (deflated 57%)
  adding: bert_tweet/config.json (deflated 50%)
  adding: bert_tweet/tokenizer_config.json (deflated 78%)


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1709644488.34334bcdfcbe.1172.1:   0%|          | 0.00/12.4k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

events.out.tfevents.1709645177.34334bcdfcbe.1172.2:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/583M [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ARC4N3/bertweet-olid/commit/21748334b3ecf4c7e9cfd3a9f83cc49fe416b91b', commit_message='End of training', commit_description='', oid='21748334b3ecf4c7e9cfd3a9f83cc49fe416b91b', pr_url=None, pr_revision=None, pr_num=None)